In [6]:
!pip install kaggle-environments -U
!cp -r ../input/lux-ai-2021/* .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.3 MB/s eta 0:00:00:00:01
  Created wheel for vec-noise: filename=vec_noise-1.1.4-cp310-cp310-linux_x86_64.whl size=35255 sha256=036d286a551a6098a584420c59e58b7e2f8cc14c07f17eb0d0aacb36b71a290a
  Stored in directory: /root/.cache/pip/wheels/2b/28/e4/f710af1a5bb24bb6da3d4f65081e268ca85034b7ac0a9237fe
Successfully built vec-noise
  Attempting uninstall: gymnasium
    Found existing installation: G

### Loading deterministic agents

In [10]:
!git clone https://github.com/aaiit/lux-AI.git

Cloning into 'lux-AI'...
Username for 'https://github.com': ^C


The inputs of the model are the map of the features and the output are map of the actions as a matrix.

In [3]:
!cp lux-AI/Game/*.py .
!cp lux-AI/Game/lux/* lux/
!cp lux-AI/*.png .

cp: cannot stat 'lux-AI/Game/*.py': No such file or directory
cp: cannot stat 'lux-AI/Game/lux/*': No such file or directory
cp: cannot stat 'lux-AI/*.png': No such file or directory


In [11]:
!mkdir snapshots
from kaggle_environments import make

mkdir: cannot create directory ‘snapshots’: File exists


In [13]:
import os
from sklearn.model_selection import train_test_split
import numpy as np 
import joblib
import pickle
import agent

In [15]:
from tensorflow.keras.utils import plot_model, Sequence
from random import random

In [16]:
# setting random state 
rs = 0

### Creating episodes 

automatically saving the game_state and actions made by the agents for futher use.

In [17]:
env = make("lux_ai_2021", debug = True, configuration = {"annotations" : True,
                                                        "width" : 12,
                                                        "height" : 12})
steps = env.run(['agent.py', 'simple_agent'])


In [18]:
env.render(mode='ipython', width=400, height=400)

In [22]:
sn = os.listdir("../input/lux-ai-900-episodes-12x12/snapshots12/snapshots")

game_state = [i for i in sn if "game_state" in i]
actions_ = [i for i in sn if "actions" in i]

len(actions_)

### Splitting Train and Test Set

In [ ]:
game_state_train, game_state_test, actions_train, actions_test = train_test_split(game_state_, actions_, test_size = 0.2, random_state = rs)

In [ ]:
str_step = game_state_[0].split('-')[1].split(".")[0]

with open(f'../input/lux-ai-900-episodes-12x12/snapshots12/snapshots/game_state-{str_step}.pkl', 'rb') as handle:
    game_state = pickle.load(handle)
    

with open(f'../input/lux-ai-900-episodes-12x12/snapshots12/snapshots/actions_as_Matrix-{str_step}.pkl', 'rb') as handle:
    Y = pickle.load(handle)

### Game State to inputs Matrix

In [ ]:
def get_inputs(game_state):
    # the shape of the map
    w,h = game_state.map.width, game_state.map.height
    
    # map of the resources 
    M = [ [0 if game_state.map.map[j][i].resources == None else game_state.map.map[j][i].resouce.amount for i in range(w)] for j in range(h) ]
    # Map of the unit features 
    U_player = [ [[0,0,0,0,0] for i in range(w)] for j in range(h) ]
    
    units = game_state.player.units
    
    for i in units:
        U_player[i.pos.y][i.pos.x] = [i.type, i.cooldown, i.cargo.wood,
                                      i.cargo.coal, i.cargo.uranium]
        
        U_player = np.array(U_player)
        
        U_opponent = [ [[0,0,0,0,0] for i in range(w)] for j in range(h)]
        
        units = game_state.opponent.units
        
        for i in units:
            U_opponent[i.pos.y][i.pos.x] = [i.type,i.cooldown,i.cargo.wood,
                                           i.cargo.coal,i.cargo.uranium]
            
            U_opponent = np.array(U_opponent)
            
            # map of the cities features
            e = game_state.player.cities
            C_player = [ [ [0,0,0] for i in range(w)] for j in range(h)]
            
            for k in e:
                citytiles = e[k].citytiles
                for i in citytiles:
                    C_player[i.pos.y][i.pos.x] = [i.cooldown,e[k].fuel,
                                                 e[k].light_upkeep]
                    C_player = np.array(C_player)
                    
                    e = game_state.opponent.cities
                    C_opponent = [[[0,0,0] for i in range(w)] for j in range(h)]
                    
                    for k in e:
                        citytiles = e[k].citytiles
                        for i in citytiles:
                            C_opponent[i.pos.y][i.pos.x] = [i.cooldown,
                                                           e[k].fuel, e[k].light_upkeep]
                            C_opponent = np.array(C_opponent)
                            
                            # stacking all in one array
                            E = np.dstack([M,U_opponent, U_player, C_opponent, C_player])
                            
                    

In [ ]:
X = get_inputs(game_state)

X.shape

### Data generators

In [ ]:
class DataGenerators(Sequence):
    """Generates data for Keras"""
    
    def __init__(self, game_state_paths, actions_paths, dir_path, batch_size=300, shuffle = True):
        """Initialization"""
        self.game_state_paths = game_state_paths
        self.actions_paths = actions_paths
        self.dir_path = dir_path
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.game_state_paths) / self.batch_size))
    
    def __getitem__(self, index):
        """Generate one batch of data"""
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        list_IDs_temp = [k for k in indexes]
        
        # Generate data 
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y
    
    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.actions_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        """Generates data containing batch_size samples"""
        batch_game_state = list()
        batch_actions = list()
        
        for i in list_IDs_temp:
            with open(self.dir_path+self.game_state_paths[i], 'rb') as handle:
                game_state = pickle.load(handle)
            with open(self.dir_path+self.game_state_paths[i], 'rb') as handle:
                y = pickle.load(handle)
                
            x = get_inputs(game_state)
            batch_game_state.append(x)
            batch_actions.append(y)
        batch_game_state = np.array(batch_game_state, dtype = np.float32)
        
        return batch_game_state, np.array(batch_actions, dtype = np.float32)
    

In [ ]:
train_generators = DataGenerator(game_state_train, actions_train,
                                 '../input/lux-ai-900-episodes-12x12/snapshots12/snapshots/', batch_size = 500, shuffle = True)
train_steps = train_generator.__len__()
train_steps